In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
import functools as ft

import anndata as ad
import numpy as np
import pandas as pd
import s3fs
import scanpy as sc

import celltrip

os.environ['AWS_PROFILE'] = 'waisman-admin'
s3 = s3fs.S3FileSystem(skip_instance_cache=True)


# Reading Files

In [4]:
# Read data
fnames = ['../data/scglue/Chen-2019-RNA.h5ad', '../data/scglue/Chen-2019-ATAC.h5ad']
adatas = celltrip.utility.processing.read_adatas(*fnames, backed=True)
celltrip.utility.processing.test_adatas(*adatas)

# Sample data
dataloader = celltrip.utility.processing.PreprocessFromAnnData(*adatas)
modalities, adata_obs, adata_vars = dataloader.sample()


In [ ]:
import time
start = time.perf_counter()
for _ in range(10): dataloader.sample()
print(f'Sampling takes ~{(time.perf_counter()-start)/10:.2f} seconds')


Sampling takes ~5.35 seconds


# Reading Large Files

In [3]:
# Read data
# NOTE: Each file still takes >1Gb for whatever reason
fnames = [f'../data/tahoe/plate{i}_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab.h5ad' for i in range(1, 15)][:4]
partition_cols = ['sample', 'plate']
adatas = celltrip.utility.processing.read_adatas(*fnames, backed=True)
adatas = celltrip.utility.processing.merge_adatas(*adatas, backed=True)
celltrip.utility.processing.test_adatas(*adatas)

# Sample data
dataloader = celltrip.utility.processing.PreprocessFromAnnData(
    *adatas, partition_cols=partition_cols)
modalities, adata_obs, adata_vars = dataloader.sample()


In [4]:
import time
start = time.perf_counter()
for _ in range(10): dataloader.sample()
print(f'Sampling takes ~{(time.perf_counter()-start)/10:.2f} seconds')


Sampling takes ~11.80 seconds


# Formatting CSV Files

In [ ]:
# Prerequisites
fnames = ['../data/MERFISH/s3_mapped_cell_table.csv', '../data/MERFISH/s3_cell_by_gene.csv']
outfiles = ['../data/MERFISH/spatial.h5ad', '../data/MERFISH/expression.h5ad']
partition_cols = 'experiment'
adatas = []

# Spatial
fname = fnames[0]
df = pd.read_csv(fname, index_col=0, header=0).set_index('sample_name')
df_obs = df[['area', 'experiment', 'layer']]
df = df[['xc_adjusted', 'yc_adjusted']]
adata = ad.AnnData(df, obs=df_obs)
adata.write_h5ad(outfiles[0])

# Gene expression
fname = fnames[1]
df = pd.read_csv(fname, index_col=0, header=1)
df.index.name = 'sample_name'
adata = ad.AnnData(df, obs=df_obs)
adata.write_h5ad(outfiles[1])


/home/thema/miniconda3/envs/celltrip/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/thema/miniconda3/envs/celltrip/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [ ]:
# Read
fnames = ['../data/MERFISH/expression.h5ad', '../data/MERFISH/spatial.h5ad']
partition_cols = 'layer'
adatas = celltrip.utility.processing.read_adatas(*fnames, backed=False)
celltrip.utility.processing.test_adatas(*adatas, partition_cols=partition_cols)

# Dataloader
dataloader = celltrip.utility.processing.PreprocessFromAnnData(
    *adatas, partition_cols=partition_cols, pca_dim=128)  # , mask=.8, seed=42
modalities, adata_obs, adata_vars = dataloader.sample()


/home/thema/repos/inept/celltrip/utility/processing.py:109: RuntimeWarning: Modality 1 too small for PCA (2 features), skipping
  warnings.warn(


# Other Files

In [3]:
# Temporal Brain
import numpy as np
import pandas as pd
import rds2py
fnames = ['../data/temporalBrain/GSE204683_count_matrix.RDS', '../data/temporalBrain/GSE204682_count_matrix.RDS']
barcodes = ['../data/temporalBrain/GSE204683_barcodes.tsv', '../data/temporalBrain/GSE204682_barcodes.tsv']
outfiles = ['../data/temporalBrain/expression.h5ad', '../data/temporalBrain/peaks.h5ad']

# Load RNA
rdata = rds2py.parse_rds(fnames[0])
rdata['attributes']['dimnames'] = rdata['attributes']['Dimnames']
wrapper = rds2py.generics._dispatcher(rdata)
M1, F1, C1 = wrapper.matrix.T, *[np.array(sl) for sl in wrapper.dimnames]
B1 = pd.read_csv(barcodes[0], delimiter='\t')

# Load ATAC
rdata = rds2py.parse_rds(fnames[1])
rdata['attributes']['dimnames'] = rdata['attributes']['Dimnames']
wrapper = rds2py.generics._dispatcher(rdata)
M2, F2, C2 = wrapper.matrix.T, *[np.array(sl) for sl in wrapper.dimnames]
B2 = pd.read_csv(barcodes[1], delimiter='\t')

# Get sample ids
uniq_col, count_col = np.unique([e.split('_')[0] for e in C1], return_counts=True)
assert np.unique(count_col, return_counts=True)[1].max() == 1  # No duplicate counts, otherwise manual annotation is needed
# Get donor ids
uniq_donor, count_donor = np.unique(B1['Donor ID'], return_counts=True)
assert (np.sort(count_col) == np.sort(count_donor)).all()
# Convert (aided by `preprocessing.R`)
name_to_id = {d: c for c, d in zip(uniq_col[np.argsort(count_col)], uniq_donor[np.argsort(count_donor)])}
# Set indices
B1['Cell ID'] = B1.apply(lambda r: f'{name_to_id[r["Donor ID"]]}_{r["Barcode"]}', axis=1)
B2['Cell ID'] = B2.apply(lambda r: f'{name_to_id[r["Donor ID"]]}_{r["Barcode"]}', axis=1)
assert (B1.set_index('Cell ID') == B1.set_index('Cell ID').loc[C1]).all().all()  # For some reason, `barcodes2` doesn't line up with `C2`, so we assume both meta are the correct order

# RNA AnnData
adata = ad.AnnData(M1, obs=B1.set_index('Cell ID').loc[C1])
adata.var_names = F1
adata.write_h5ad(outfiles[0])

# ATAC AnnData
adata = ad.AnnData(M2, obs=B2.set_index('Cell ID'))
adata.var_names = F2
adata.write_h5ad(outfiles[1])


# Upload to s3

In [6]:
# Aggregate files to upload
fnames = []; folders = []

# scGLUE
# fnames += ['../data/scglue/Chen-2019-RNA.h5ad', '../data/scglue/Chen-2019-ATAC.h5ad']
# folders += 2*['scGLUE']

# MERFISH
# fnames += ['../data/MERFISH/expression.h5ad', '../data/MERFISH/spatial.h5ad']
# folders += 2*['MERFISH']

# Temporal Brain
fnames += ['../data/temporalBrain/expression.h5ad', '../data/temporalBrain/peaks.h5ad']
folders += 2*['TemporalBrain']

# TAHOE-100M
# fnames = [f'../data/tahoe/plate{i}_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab.h5ad' for i in range(1, 15)]
# folders += 2*['Tahoe']

# Upload
for folder, fname in zip(folders, fnames): s3.put(fname, os.path.join(os.path.join('s3://nkalafut-celltrip', folder), os.path.basename(fname)))


# Perturb or Knock Features in Processed Space

In [ ]:
# Perform gene knockdown in processed data
modality_to_test = 0
features_to_test = [200, 150]
adatas, _, adata_vars = dataloader.get_transformables()
iso_modality = dataloader.preprocessing.transform(
    adatas[modality_to_test][adata_obs[modality_to_test].index].X,
    # force_filter=True,
    subset_features=features_to_test,
    subset_modality=modality_to_test)[0]
knocked_modality = modalities[modality_to_test] - iso_modality

# Verify the knockdown works
orig_modality, = dataloader.preprocessing.inverse_transform(modalities[modality_to_test], subset_modality=modality_to_test)
inv_modality, = dataloader.preprocessing.inverse_transform(knocked_modality, subset_modality=modality_to_test)
change = np.abs(orig_modality - inv_modality).sum(axis=0) / dataloader.preprocessing.standardize_std[modality_to_test]
most_changed_idx = dataloader.preprocessing.filter_mask[modality_to_test][np.argsort(change).flatten()[-len(features_to_test):]]
print(f'Targets: {np.array(features_to_test)}, Most Changed: {most_changed_idx}, should be the same elements')


Targets: [200 150], Most Changed: [150 200], should be the same elements


In [ ]:
import time
start = time.perf_counter()
for _ in range(10): dataloader.sample()
print(f'Sampling takes ~{(time.perf_counter()-start)/10:.2f} seconds')


Sampling takes ~0.00 seconds
